In [3]:
import os
import time
import numpy as np
import sys
import torch
import torch.optim as optim
sys.path.append('..')
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
from cogito.machine import MachineConfig
from agora.Non_DAG.algorithm_energy.random_algorithm import RandomAlgorithm
from agora.Non_DAG.algorithm_energy.tetris import Tetris
from agora.Non_DAG.algorithm_energy.first_fit import FirstFitAlgorithm
from agora.Non_DAG.algorithm.PPO_Energy.PPO_classic import RLAlgorithm
from agora.Non_DAG.algorithm.PPO_Energy.Agent import Agent
# from agora.Non_DAG.algorithm.PPO.Networks import brain

from agora.Non_DAG.algorithm.DeepJS.reward_giver import AverageCompletionRewardGiver, AverageSlowDownRewardGiver, \
    MakespanRewardGiver
# from agora.Non_DAG.algorithm.PPO.reward_giver import EnergyOptimisationRewardGiver

from agora.Non_DAG.utils.csv_reader import CSVReader
from agora.Non_DAG.utils.feature_functions import features_extract_func_ac, features_normalize_func_ac
from agora.Non_DAG.utils.tools import multiprocessing_run, average_completion, average_slowdown
from agora.Non_DAG.utils.episode import Episode
from agora.DAG.algorithm.gnnDeepRL.reward_giver import EnergyBaseline,CombinedReward,MakespanRewardGiver,Energy,EnergyOptimisationTask, EnergyOptimisationRewardGiverV2,EnergyOptimisationRewardGiver

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

np.random.seed(41)

# ************************ Parameters Setting Start ************************
machines_number = 5
jobs_len = 1
n_iter = 1
jobs_csv = '../jobs_files/jobs.csv'

reward_giver = EnergyOptimisationRewardGiverV2()

name = f'GNN-{machines_number}'
model_dir = f'./agents/{name}'
summary_path = f'./Tensorboard/{name}'

# ************************ Parameters Setting End ************************

if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

csv_reader = CSVReader(jobs_csv)

os.environ['CUDA_VISIBLE_DEVICES'] = ''
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
np.random.seed(1)
# tf.random.set_random_seed(41)
# ************************ Parameters Setting Start *************w***********
machines_number = 5
jobs_len =1
n_iter = 150
n_episode = 12
jobs_csv = '../jobs_files/jobs.csv'

# brain = brain(9)
# reward_giver = AverageCompletionRewardGiver()
#reward_giver = AverageSlowDownRewardGiver()
energy_giver = Energy()
makespan_giver = MakespanRewardGiver(reward_per_timestamp=1.0)
#reward_giver = CombinedReward(energy_giver, makespan_giver)
reward_giver =energy_giver

# reward_giver = EnergyOptimisationRewardGiver()
features_extract_func = features_extract_func_ac
features_normalize_func = features_normalize_func_ac

# name = '%s-%s-m%d' % (reward_giver.name, brain.name, machines_number)
# model_dir = './agents/%s' % name
# ************************ Parameters Setting End ************************

# if not os.path.isdir(model_dir):
#    os.makedirs(model_dir)

"**********************************************************"


def calculate_mean(list_of_lists):
    total_sum = 0
    total_count = 0

    for sublist in list_of_lists:
        total_sum += sum(sublist)
        total_count += len(sublist)

    m = total_sum / total_count
    return m


"*************************************************************"
gamma=0.99
epsilon=0.2
entropy_coefficient=0.15
lr=0.001
batch_size=1000
csv_reader = CSVReader(jobs_csv)
machine_configs = [
    MachineConfig(0, 240, 1, 1, "ThinkSystem-SR850-V3", 1900),
    MachineConfig(1, 200, 2, 1, "HPProLiant-ML30-Gen11", 3200),
    MachineConfig(2, 512, 2, 1, "FALINUX-AnyStor-700EC-NM", 3000),
    MachineConfig(4, 120, 1, 1, "FusionServer-1288H-V7", 1900),
    MachineConfig(3, 512, 5, 1, "Dell-PowerEdgeHS5610", 2100),
    # Add other entries as required
] * 1 # Repeat the list multiple times as shown in your example

# Update the first parameter of each MachineConfig iteratively
for i, config in enumerate(machine_configs):
    config.param1 = i  # Assuming param1 is the first attribute in the MachineConfig class

epsilon = 0.99 # Start with full exploration
epsilon_min = 0.01  # Minimum exploration
epsilon_decay = 0.99  # Epsilon decay rate per step

Creating
Creating


In [4]:
for job_chunk in range(0,10):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for j in jobs_configs:
        print(j.submit_time)
        print(j.tasks_map)
        for k,task_config in j.tasks_map.items():
            task_config_cpu=task_config.cpu
            task_config_memory=task_config.memory
            task_config_duration=task_config.duration
            
        

Jobs number:  1
Tasks number: 11
Task instances number mean:  84.9090909090909
Task instances number std 79.68398327193893
Task instances cpu mean:  0.5
Task instances cpu std:  0.0
Task instances memory mean:  0.009669336069822105
Task instances memory std:  0.0017946966733641452
Task instances duration mean:  97.95074946466808
Task instances duration std:  37.77646693750588
0.0
{'9550.0': <cogito.config.TaskConfig object at 0x000001B0B4324ED0>, '9551.0': <cogito.config.TaskConfig object at 0x000001B0B4324DD0>, '9552.0': <cogito.config.TaskConfig object at 0x000001B0B4325410>, '9553.0': <cogito.config.TaskConfig object at 0x000001B0B43253D0>, '9554.0': <cogito.config.TaskConfig object at 0x000001B0B4325010>, '9555.0': <cogito.config.TaskConfig object at 0x000001B0B4325510>, '9556.0': <cogito.config.TaskConfig object at 0x000001B0B4325910>, '9557.0': <cogito.config.TaskConfig object at 0x000001B0B4325610>, '9558.0': <cogito.config.TaskConfig object at 0x000001B0B4325590>, '9559.0': <co

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

for job_chunk in range(0,10):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for job_index, j in enumerate(jobs_configs):
        # Create a list to store task data
        task_data = []

        for task_id, task_config in j.tasks_map.items():
            task_data.append({
                'Task ID': task_id,
                'CPU': task_config.cpu,
                'Memory': task_config.memory,
                'Duration': task_config.duration
            })

        # Convert to DataFrame
        df = pd.DataFrame(task_data)

        # Create a figure with subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
        fig.suptitle(f'Job {job_index + 1} Task Analysis (Submit Time: {j.submit_time})')

        # 1. Bar plot for CPU requirements
        sns.barplot(x='Task ID', y='CPU', data=df, ax=ax1)
        ax1.set_title('CPU Requirements by Task')
        ax1.set_xlabel('Task ID')
        ax1.set_ylabel('CPU')

        # 2. Bar plot for Memory requirements
        sns.barplot(x='Task ID', y='Memory', data=df, ax=ax2)
        ax2.set_title('Memory Requirements by Task')
        ax2.set_xlabel('Task ID')
        ax2.set_ylabel('Memory')

        # 3. Scatter plot for CPU vs Memory, with size representing Duration
        sns.scatterplot(x='CPU', y='Memory', size='Duration', data=df, ax=ax3)
        ax3.set_title('CPU vs Memory (Size: Duration)')
        ax3.set_xlabel('CPU')
        ax3.set_ylabel('Memory')

        plt.tight_layout()
        
        # Save the plot
        plt.savefig(f'job_{job_index + 1}_analysis.png')
        plt.close()

    print(f"Plots for job chunk {job_chunk} have been saved.")

Length of job_configs is  5
Jobs number:  5
Tasks number: 10
Task instances number mean:  54.7
Task instances number std 154.52446408255233
Task instances cpu mean:  52.376599634369285
Task instances cpu std:  21.88178395095497
Task instances memory mean:  0.5850822669104205
Task instances memory std:  0.06403368768881976
Task instances duration mean:  25.424131627056674
Task instances duration std:  8.37005211013451
Plots for job chunk 4 have been saved.


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

for job_chunk in range(0,10):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for job_index, j in enumerate(jobs_configs):
        task_data = []

        for task_id, task_config in j.tasks_map.items():
            task_data.append({
                'Task ID': task_id,
                'CPU': task_config.cpu,
                'Memory': task_config.memory,
                'Duration': task_config.duration
            })

        df = pd.DataFrame(task_data)

        # Create a figure with subplots
        fig = plt.figure(figsize=(20, 25))
        fig.suptitle(f'Job {job_index + 1} Task Analysis (Submit Time: {j.submit_time})', fontsize=16)

        # 1. Bar plot for CPU requirements
        ax1 = fig.add_subplot(421)
        sns.barplot(x='Task ID', y='CPU', data=df, ax=ax1)
        ax1.set_title('CPU Requirements by Task')
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')

        # 2. Bar plot for Memory requirements
        ax2 = fig.add_subplot(422)
        sns.barplot(x='Task ID', y='Memory', data=df, ax=ax2)
        ax2.set_title('Memory Requirements by Task')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')

        # 3. Scatter plot for CPU vs Memory, with size representing Duration
        ax3 = fig.add_subplot(423)
        sns.scatterplot(x='CPU', y='Memory', size='Duration', hue='Duration', data=df, ax=ax3)
        ax3.set_title('CPU vs Memory (Size/Color: Duration)')

        # 4. Histogram of CPU requirements
        ax4 = fig.add_subplot(424)
        sns.histplot(data=df, x='CPU', kde=True, ax=ax4)
        ax4.set_title('Distribution of CPU Requirements')

        # 5. Histogram of Memory requirements
        ax5 = fig.add_subplot(425)
        sns.histplot(data=df, x='Memory', kde=True, ax=ax5)
        ax5.set_title('Distribution of Memory Requirements')

        # 6. Box plot of CPU and Memory
        ax6 = fig.add_subplot(426)
        df_melted = pd.melt(df, value_vars=['CPU', 'Memory'])
        sns.boxplot(x='variable', y='value', data=df_melted, ax=ax6)
        ax6.set_title('Box Plot: CPU and Memory')

        # 7. Heatmap of correlation between CPU, Memory, and Duration
        ax7 = fig.add_subplot(427)
        corr = df[['CPU', 'Memory', 'Duration']].corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax7)
        ax7.set_title('Correlation Heatmap')

        # 8. Parallel coordinates plot
        ax8 = fig.add_subplot(428)
        pd.plotting.parallel_coordinates(df, 'Task ID', ax=ax8)
        ax8.set_title('Parallel Coordinates Plot')
        ax8.legend_.remove()

        plt.tight_layout()
        plt.subplots_adjust(top=0.95)
        
        # Save the plot
        plt.savefig(f'job_{job_chunk}{job_index + 1}_detailed_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

    print(f"Detailed plots for job chunk {job_chunk} have been saved.")

Jobs number:  1
Tasks number: 11
Task instances number mean:  84.9090909090909
Task instances number std 79.68398327193893
Task instances cpu mean:  0.5
Task instances cpu std:  0.0
Task instances memory mean:  0.009669336069822105
Task instances memory std:  0.0017946966733641452
Task instances duration mean:  97.95074946466808
Task instances duration std:  37.77646693750588
Detailed plots for job chunk 0 have been saved.
Jobs number:  1
Tasks number: 8
Task instances number mean:  75.75
Task instances number std 55.74215191396902
Task instances cpu mean:  0.5
Task instances cpu std:  0.0
Task instances memory mean:  0.010221976304223196
Task instances memory std:  0.0023750256169184155
Task instances duration mean:  77.31848184818483
Task instances duration std:  42.88684875720535
Detailed plots for job chunk 1 have been saved.
Jobs number:  1
Tasks number: 18
Task instances number mean:  27.22222222222222
Task instances number std 27.36696214934991
Task instances cpu mean:  0.5
Task